String comparisons are complicated by the fact that Unicode has combining
characters: diacritics and other marks that attach to the preceding character,
appearing as one when printed.

For example, the word “café” may be composed in two ways, using four or
five code points, but the result looks exactly the same:

In [1]:
s1 = 'café'
s2 = 'cafe\N{COMBINING ACUTE ACCENT}'

s1, s2

('café', 'café')

In [2]:
len(s1), len(s2)


(4, 5)

In [4]:
s1 == s2

False

In [10]:
from unicodedata import normalize, name
s1 = 'café'
s2 = 'cafe\N{COMBINING ACUTE ACCENT}'
len(s1), len(s2)


(4, 5)

In [7]:
len(normalize('NFC', s1)), len(normalize('NFC', s2))

(4, 4)

In [8]:
len(normalize('NFD', s1)), len(normalize('NFD', s2))

(5, 5)

In [11]:
ohm = '\u2126'
name(ohm)

'OHM SIGN'